In [1]:
from collections import Counter
import tokenization
import preprocess
import ner
import verb_extraction
import stemming


In [3]:
singleNumbers = ["واحد","اتنين ","تلاته","اربعه","خمسه","سته","سبعه","تمانيه","تسعه"]
higherNumbers = ["عشر","عشرين","تلاتين","اربعين","خمسين"]
numbersBagOfWords = []
for number in singleNumbers:
    numbersBagOfWords.append(number)    

for higher in higherNumbers:
    if higher == "عشر":
        numbersBagOfWords.append(higher + "ه")
    else:
        numbersBagOfWords.append(higher)
    for number in singleNumbers:
        if higher == "عشر" or higher == "":
            numbersBagOfWords.append(number + higher)    
        else:
            numbersBagOfWords.append(number + "و" + higher)
print(numbersBagOfWords)

['واحد', 'اتنين ', 'تلاته', 'اربعه', 'خمسه', 'سته', 'سبعه', 'تمانيه', 'تسعه', 'عشره', 'واحدعشر', 'اتنين عشر', 'تلاتهعشر', 'اربعهعشر', 'خمسهعشر', 'ستهعشر', 'سبعهعشر', 'تمانيهعشر', 'تسعهعشر', 'عشرين', 'واحدوعشرين', 'اتنين وعشرين', 'تلاتهوعشرين', 'اربعهوعشرين', 'خمسهوعشرين', 'ستهوعشرين', 'سبعهوعشرين', 'تمانيهوعشرين', 'تسعهوعشرين', 'تلاتين', 'واحدوتلاتين', 'اتنين وتلاتين', 'تلاتهوتلاتين', 'اربعهوتلاتين', 'خمسهوتلاتين', 'ستهوتلاتين', 'سبعهوتلاتين', 'تمانيهوتلاتين', 'تسعهوتلاتين', 'اربعين', 'واحدواربعين', 'اتنين واربعين', 'تلاتهواربعين', 'اربعهواربعين', 'خمسهواربعين', 'ستهواربعين', 'سبعهواربعين', 'تمانيهواربعين', 'تسعهواربعين', 'خمسين', 'واحدوخمسين', 'اتنين وخمسين', 'تلاتهوخمسين', 'اربعهوخمسين', 'خمسهوخمسين', 'ستهوخمسين', 'سبعهوخمسين', 'تمانيهوخمسين', 'تسعهوخمسين']


In [67]:
def edit_distance (word1, word2, index1, index2):
    dp = [[0 for x in range(index2 + 1)] for x in range(index1 + 1)]
    for i in range (index1 + 1):
        for j in range (index2 + 1):
            if i == 0:
                dp[i][j] == j
            elif j == 0:
                dp[i][j] == i
            elif word1[i - 1] == word2[j - 1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                 dp[i][j] = 1 + min ( dp[i-1][j-1], #replace
                                    dp[i-1][j],     #remove
                                    dp[i][j-1])     #insert
    return dp[index1][index2]
    
def textNumberToNumber(word, numbersBagOfWords):
    if word.isnumeric():
        return int(word), 0
    editDistance = 1000
    closestNumber = "واحد"
    for index, number in enumerate(numbersBagOfWords):
        editDistanceNew = edit_distance(word, number, len(word), len(number))
        if (editDistanceNew < editDistance or 
        (editDistanceNew == editDistance and abs( len(number) - len(word)) < abs( len(closestTextualNumber) - len(word)))):
            editDistance = editDistanceNew
            closestTextualNumber = number
            closestNumber = index + 1
    if editDistance >= 3:
        return 0 , editDistance
    return closestNumber, editDistance
    
def bigrams (tokens):
    grams = []
    for index in range(len(tokens) - 1):
        grams.append([tokens[index] , tokens[index + 1]])
    return grams

def get_closest_word (word, tokens):
    min_dist = 10000
    min_index = 0
    for index, wordInText in enumerate(tokens):
        distance = edit_distance(word,wordInText, len(word), len(wordInText))
        if distance < min_dist:
            min_dist = distance
            min_index = index
    return min_index

In [107]:
def NLU(text):
    #Preprocessing
    text = preprocess.pre_process(text)
    #Tokenization
    tokens = tokenization.get_tokens(text)
    # NER
    ents = ner.get_ents(tokens)
    #Part of Speech and Stemming
    tokens_verb_noun = verb_extraction.extract_stem_verb(tokens,ents)
    tokens_verb_noun = stemming.stem(tokens_verb_noun)
    return text , tokens ,ents , tokens_verb_noun
text = input()
text, tokens, ents, tokens_verb_noun = NLU(text)

filtered_tokens = []
for index, token in enumerate(tokens):
    if tokens_verb_noun[index][1] != "vOrder":
        filtered_tokens.append(token)

for index, token in enumerate(filtered_tokens):
    if token[0] == 'ا' and token[1] == 'ل':
        filtered_tokens[index] = token[2:]
    elif token[0] == 'و' :
        filtered_tokens[index] = token[1:]
print(text)

اظبطلي المنبه 4 الفجر بكره


c:\Users\Ahmed Sherif\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\Users\Ahmed Sherif\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LinearSVC from version 0.23.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\Users\Ahmed Sherif\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [108]:
bigram = bigrams(filtered_tokens)
bigram.reverse()
# minuteEdit, hourEdit, dayEdit, monthEdit, overwriteHour, overwriteDay, overwriteMonth, at night
edits = [0, 0, 0, 0, 0, 0, 0, 1]
# specifier -> [check index in edits, how much to add]
time_specifiers_second = {
    "ساعه" : [1, 1], "يوم" : [2, 1], "شهر" : [3, 1], "بكره" : [2, 1],
    "ساعتين" : [1, 2], "يومين" : [2, 2], "شهرين" : [3, 2],
    "ساعات" : [1, 3], "ايام" : [2, 3], "شهور" : [3, 3],
    "ربع" : [0, 15], "تلت" : [0, 20], "نص" : [0, 30],
    "دقايق" : [0, 3], "دقيقه" : [0, 3]
}
time_specifiers_first = {
    "ساعه" : [1], "يوم" : [2], "شهر" : [3]
}
tokens_used = [0] * len(filtered_tokens)
for index, gram in enumerate(bigram):
    if gram[0] in time_specifiers_first :
        specifier = time_specifiers_first.get(gram[0])
        if edits[specifier[0]] == 0:
            edits[specifier[0]], _ = textNumberToNumber(gram[1], numbersBagOfWords)
            edits[specifier[0] + 3] = 1
            tokens_used[filtered_tokens.index(gram[0])] = 1
            tokens_used[filtered_tokens.index(gram[1])] = 1
    
    if gram[1] in time_specifiers_second:
        specifier = time_specifiers_second.get(gram[1])
        if edits[specifier[0]] == 0:
            edits[specifier[0]] = specifier[1]
            if specifier[1] == 3:
                edits[specifier[0]], _ = textNumberToNumber(gram[0], numbersBagOfWords) 
                tokens_used[filtered_tokens.index(gram[0])] = 1
            tokens_used[filtered_tokens.index(gram[1])] = 1


# no specifiers
if edits[1] == 0:            
    minDistance = 1000
    token_index = -1
    for index, token in enumerate(filtered_tokens):
        if tokens_used[index] == 0:
            hoursEdit, distance = textNumberToNumber(token, numbersBagOfWords)
            if hoursEdit != 0 and distance < minDistance:
                minDistance = distance
                edits[1] = hoursEdit
                edits[4] = 1
                token_index = index
    if token_index != -1:
        tokens_used[token_index] = 1

time_specifiers_tokens = {
    "صبح" : [0], "ليل" : [1], "صباح" : [0], "نهار" : [1], "ظهر" : [1], "عصر" : [1], "مغرب" : [1] ,
    "عشاء" : [1], "عشا" : [1], "فجر" : [0]
}
for token in filtered_tokens:
    if token in time_specifiers_tokens:
        tokens_used[filtered_tokens.index(token)] = 1
        edits[7] = time_specifiers_tokens.get(token)[0]


            

print (edits, tokens_used, filtered_tokens)

longest_subsequence, longest_subsequence_start, start_index, last_index = -1, 0, 0, 0
for index, used_token in enumerate(tokens_used):
    if used_token == 0:
        last_index += 1
    else:
        if last_index - start_index > longest_subsequence :
            longest_subsequence = last_index - start_index
            longest_subsequence_start = start_index
        start_index, last_index =  index + 1, index + 1
if used_token == 0:
    if last_index - start_index > longest_subsequence :
            longest_subsequence = last_index - start_index
            longest_subsequence_start = start_index
        
filtered_tokens[longest_subsequence_start]
print (filtered_tokens[longest_subsequence_start], filtered_tokens[ longest_subsequence_start + longest_subsequence - 1])

text_splitted = text.split()
content = text_splitted[ get_closest_word (filtered_tokens[longest_subsequence_start], text_splitted):
 get_closest_word (filtered_tokens[ longest_subsequence_start + longest_subsequence - 1], text_splitted) + 1]
content = " ".join(content)
print(content)
# need to add "بكرة" "الصبح" "بليل" 
# cannot deiffrentiate between "watch" and "hour" both are "ساعة"
# it is a static code any error will result into a distaster XD


[0, 4, 1, 0, 1, 0, 0, 0] [0, 0, 1, 1, 1] ['اظبطلي', 'منبه', '4', 'فجر', 'بكره']
اظبطلي منبه
اظبطلي المنبه


In [109]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

now_time = datetime.now()
print(now_time)
new_time = datetime(year= now_time.year
                    , month= ( edits[3] * edits[6] ) + ( now_time.month * (not edits[6]) )
                    , day=   ( edits[2] * edits[5] ) + ( now_time.day * (not edits[5]) )
                    , hour=  ( edits[1] * edits[4] + 12 * edits[7]) + ( now_time.hour * (not edits[4]) )
                    , minute=( edits[0] * edits[4] ) + ( now_time.minute * (not edits[4]) ))
print(new_time)
new_time = new_time + relativedelta(minutes= edits[0] * (not edits[4]), hours=edits[1] * (not edits[4]), days=edits[2] * (not edits[5]), months=edits[3] * (not edits[6]))
print(new_time)

2022-07-03 18:17:23.107417
2022-07-03 04:00:00
2022-07-04 04:00:00
